In [4]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 22 07:05:30 2017

@author: angusurquhart
"""

#Figure out how to loop through urls
from collections import defaultdict

import trackopy
import pandas as pd

In [5]:
#Setup trackobot
user = {"password":'e479deb2e1', 'username':'still-gnoll-6604'}
trackobot = trackopy.Trackobot(user['username'], user['password'])

In [6]:
#trackobot.history(page=7)

In [31]:
#Put all the pages into a single dictionary
#Note that this is game level data - 'history'
singledict = {}
list_of_game_dicts = []

for x in range(1,8):

     singledict[1] = trackobot.history(page=x)
     list_of_game_dicts.append(singledict.copy())
    
list_of_game_dicts

[{1: {'history': [{'added': '2017-03-20T07:54:57.000Z',
     'card_history': [],
     'coin': True,
     'duration': 777,
     'hero': 'Paladin',
     'hero_deck': None,
     'id': 67984619,
     'mode': 'casual',
     'note': None,
     'opponent': 'Shaman',
     'opponent_deck': None,
     'result': 'win'},
    {'added': '2017-03-20T07:41:49.000Z',
     'card_history': [],
     'coin': False,
     'duration': 59,
     'hero': 'Paladin',
     'hero_deck': None,
     'id': 67983922,
     'mode': 'casual',
     'note': None,
     'opponent': 'Hunter',
     'opponent_deck': None,
     'result': 'win'},
    {'added': '2017-03-19T20:26:52.000Z',
     'card_history': [],
     'coin': False,
     'duration': 651,
     'hero': 'Shaman',
     'hero_deck': 'Jade',
     'id': 67935063,
     'legend': None,
     'mode': 'ranked',
     'note': None,
     'opponent': 'Druid',
     'opponent_deck': None,
     'rank': None,
     'result': 'win'},
    {'added': '2017-03-19T10:59:14.000Z',
     'card_h

In [32]:
#this shows that i can access a single page
#I'll come back to this and loop over it once I have my output ready
page = list_of_game_dicts[0]
page

{1: {'history': [{'added': '2017-03-20T07:54:57.000Z',
    'card_history': [],
    'coin': True,
    'duration': 777,
    'hero': 'Paladin',
    'hero_deck': None,
    'id': 67984619,
    'mode': 'casual',
    'note': None,
    'opponent': 'Shaman',
    'opponent_deck': None,
    'result': 'win'},
   {'added': '2017-03-20T07:41:49.000Z',
    'card_history': [],
    'coin': False,
    'duration': 59,
    'hero': 'Paladin',
    'hero_deck': None,
    'id': 67983922,
    'mode': 'casual',
    'note': None,
    'opponent': 'Hunter',
    'opponent_deck': None,
    'result': 'win'},
   {'added': '2017-03-19T20:26:52.000Z',
    'card_history': [],
    'coin': False,
    'duration': 651,
    'hero': 'Shaman',
    'hero_deck': 'Jade',
    'id': 67935063,
    'legend': None,
    'mode': 'ranked',
    'note': None,
    'opponent': 'Druid',
    'opponent_deck': None,
    'rank': None,
    'result': 'win'},
   {'added': '2017-03-19T10:59:14.000Z',
    'card_history': [],
    'coin': False,
    'dur

In [17]:
#this shows the level at which useful data is containted
#the meta stuff in this case is pretty useless - the real info is in the history.
page[1].keys()

dict_keys(['history', 'meta'])

In [18]:
#This is actually the data we'll want to flatten into a table
page[1]['history']

[{'added': '2017-03-20T07:54:57.000Z',
  'card_history': [],
  'coin': True,
  'duration': 777,
  'hero': 'Paladin',
  'hero_deck': None,
  'id': 67984619,
  'mode': 'casual',
  'note': None,
  'opponent': 'Shaman',
  'opponent_deck': None,
  'result': 'win'},
 {'added': '2017-03-20T07:41:49.000Z',
  'card_history': [],
  'coin': False,
  'duration': 59,
  'hero': 'Paladin',
  'hero_deck': None,
  'id': 67983922,
  'mode': 'casual',
  'note': None,
  'opponent': 'Hunter',
  'opponent_deck': None,
  'result': 'win'},
 {'added': '2017-03-19T20:26:52.000Z',
  'card_history': [],
  'coin': False,
  'duration': 651,
  'hero': 'Shaman',
  'hero_deck': 'Jade',
  'id': 67935063,
  'legend': None,
  'mode': 'ranked',
  'note': None,
  'opponent': 'Druid',
  'opponent_deck': None,
  'rank': None,
  'result': 'win'},
 {'added': '2017-03-19T10:59:14.000Z',
  'card_history': [],
  'coin': False,
  'duration': 749,
  'hero': 'Druid',
  'hero_deck': None,
  'id': 67863615,
  'legend': None,
  'mode':

In [35]:
#This is the meta data for a single game
page[1]['history'][0]

{'added': '2017-03-20T07:54:57.000Z',
 'card_history': [],
 'coin': True,
 'duration': 777,
 'hero': 'Paladin',
 'hero_deck': None,
 'id': 67984619,
 'mode': 'casual',
 'note': None,
 'opponent': 'Shaman',
 'opponent_deck': None,
 'result': 'win'}

In [28]:
#This is the id for a single game
page[1]['history'][0]['id']

67983922

In [60]:
#Build a dataframe of page 1 meta info
df = pd.DataFrame(page[1]['history'], columns=['added','coin', 'duration','hero','hero_deck','id','legend','mode','note','opponent','opponent_deck','rank','result'])
df

,added,coin,duration,hero,hero_deck,id,legend,mode,note,opponent,opponent_deck,rank,result
0,2017-03-20T07:54:57.000Z,True,777,Paladin,None,67984619,NaN,casual,None,Shaman,None,NaN,win
1,2017-03-20T07:41:49.000Z,False,59,Paladin,None,67983922,NaN,casual,None,Hunter,None,NaN,win
2,2017-03-19T20:26:52.000Z,False,651,Shaman,Jade,67935063,NaN,ranked,None,Druid,None,NaN,win
3,2017-03-19T10:59:14.000Z,False,749,Druid,None,67863615,NaN,ranked,None,Druid,None,NaN,win
4,2017-03-19T08:30:31.000Z,False,571,Shaman,Control,67850295,NaN,ranked,None,Priest,Dragon,NaN,win
5,2017-03-19T08:20:44.000Z,True,530,Shaman,Jade,67849592,NaN,ranked,None,Warlock,Zoo,15,win
6,2017-03-19T08:11:47.000Z,True,432,Shaman,Jade,67849006,NaN,ranked,None,Paladin,Murloc,15,win
7,2017-03-17T22:22:29.000Z,True,708,Warlock,Reno,67674426,NaN,ranked,None,Shaman,Jade,15,loss
8,2017-03-17T22:10:29.000Z,True,551,Warlock,Reno,67673214,NaN,ranked,None,Shaman,Aggro,15,win
9,2017-03-17T22:01:06.000Z,False,363,Warlock,Reno,67672209,NaN,ranked,None,Shaman,Jade,NaN,loss


In [40]:
#Now i need to build this table for every page

for i in range(len('test')):
    list_of_game_dicts[i][1]['history']


In [61]:
#Create an empty dataframe for all the meta game fields
columns = ['added','coin', 'duration','hero','hero_deck','id','legend','mode','note','opponent','opponent_deck','rank','result']
metatable = pd.DataFrame(columns=columns)
metatable

,added,coin,duration,hero,hero_deck,id,legend,mode,note,opponent,opponent_deck,rank,result


In [55]:
#append on the first of the meta game pages

all_meta_table = metatable.append(pd.DataFrame(page[1]['history'], columns=['added','coin', 'duration','hero','hero_deck','id','legend','mode','note','opponent','opponent_deck','rank','result']),ignore_index = True)



In [63]:
#test to access a table from the top level
list_of_game_dicts[0][1]['history'][0]['id']

67984619